In [1]:
import pandas as pd
from sklearn import cross_validation, svm, preprocessing, pipeline
from helpers import load_titanic

In [2]:
# load the titanic data
train, test = load_titanic()

In [3]:
# create feature vectors and labels
features = ['Sex', 'Pclass', 'Age', 'Fare', 'SibSp', 'Parch', 'Embarked']
X = train[features].values
y = train['Survived'].values

In [4]:
# create the classifier
clf = pipeline.make_pipeline(preprocessing.StandardScaler(), svm.LinearSVC())

# cross validate on the training data
scores = cross_validation.cross_val_score(clf, X, y, cv=5)
print(scores)

[ 0.78212291  0.7877095   0.78089888  0.75280899  0.81355932]


In [5]:
# predict on testing data
clf.fit(X, y)
test['Survived'] = clf.predict(test[features])

In [6]:
# write predictions to csv
pred_file = '../data/svm.csv'
test[['PassengerId', 'Survived']].to_csv(pred_file, index=False)